In [5]:
# Dependencies
import requests
import json
from datetime import datetime
from sodapy import Socrata
import pandas as pd
from config import api_key, fromdate, todate
from opendata_config.py import token311, username, password


ModuleNotFoundError: No module named 'config'

##### In the following script we'll try to analize the following

###### NYC311 can provide you access to non-emergency City services and information about City government programs. NYC311 can help with a broad range of services, including things like homeless person assistance, pothole repair, or help understanding a property tax bill. over 500 services

###### - which are the top 10 most popular report ploblems 311 receives (top 3 apartment mainrenance complaint, ilegal parking and noise from neighbor)
###### - what's the time gap they take to solve problems 
###### - what location report more issues in nyc 
###### - What burough has the highest level of 311 calls
###### - Were the 311 calls resolved (which burough has the highest resolution rate)
###### - Which category of service is the best/worst

In [20]:
# URL for GET requests to retrieve vehicle data
url_calendar= 'https://api.nyc.gov/public/api/GetCalendar?fromdate={fromdate}&todate={todate}'
headers = {'Ocp-Apim-Subscription-Key': api_key}

In [31]:
# Print the response object to the console
response = requests.get(url_calendar, headers=headers)
print(response.status_code)

200


In [32]:
# Retrieving data and converting it into JSON
response_json =response.json()

In [33]:
# Pretty Print the output of the JSON
print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "days": [
        {
            "items": [
                {
                    "details": "Alternate side parking and meters are in effect.",
                    "status": "IN EFFECT",
                    "type": "Alternate Side Parking"
                },
                {
                    "details": "Trash, recycling, and compost collections are on schedule.",
                    "status": "ON SCHEDULE",
                    "type": "Collections"
                },
                {
                    "details": "Public schools are open.",
                    "status": "OPEN",
                    "type": "Schools"
                }
            ],
            "today_id": "20240131"
        },
        {
            "items": [
                {
                    "details": "Alternate side parking and meters are in effect.",
                    "status": "IN EFFECT",
                    "type": "Alternate Side Parking"
                },
                {
                   

In [38]:
# URL for GET requests to retrieve vehicle data
url_service = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$query=SELECT%0A%20%20%60unique_key%60%2C%0A%20%20%60created_date%60%2C%0A%20%20%60closed_date%60%2C%0A%20%20%60agency%60%2C%0A%20%20%60agency_name%60%2C%0A%20%20%60complaint_type%60%2C%0A%20%20%60descriptor%60%2C%0A%20%20%60location_type%60%2C%0A%20%20%60incident_zip%60%2C%0A%20%20%60incident_address%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60cross_street_1%60%2C%0A%20%20%60cross_street_2%60%2C%0A%20%20%60intersection_street_1%60%2C%0A%20%20%60intersection_street_2%60%2C%0A%20%20%60address_type%60%2C%0A%20%20%60city%60%2C%0A%20%20%60landmark%60%2C%0A%20%20%60facility_type%60%2C%0A%20%20%60status%60%2C%0A%20%20%60due_date%60%2C%0A%20%20%60resolution_description%60%2C%0A%20%20%60resolution_action_updated_date%60%2C%0A%20%20%60community_board%60%2C%0A%20%20%60bbl%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60x_coordinate_state_plane%60%2C%0A%20%20%60y_coordinate_state_plane%60%2C%0A%20%20%60open_data_channel_type%60%2C%0A%20%20%60park_facility_name%60%2C%0A%20%20%60park_borough%60%2C%0A%20%20%60vehicle_type%60%2C%0A%20%20%60taxi_company_borough%60%2C%0A%20%20%60taxi_pick_up_location%60%2C%0A%20%20%60bridge_highway_name%60%2C%0A%20%20%60bridge_highway_direction%60%2C%0A%20%20%60road_ramp%60%2C%0A%20%20%60bridge_highway_segment%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%2C%0A%20%20%60%3A%40computed_region_efsh_h5xi%60%2C%0A%20%20%60%3A%40computed_region_f5dn_yrer%60%2C%0A%20%20%60%3A%40computed_region_yeji_bk3q%60%2C%0A%20%20%60%3A%40computed_region_92fq_4b7q%60%2C%0A%20%20%60%3A%40computed_region_sbqj_enih%60%2C%0A%20%20%60%3A%40computed_region_7mpf_4k6g%60%0AORDER%20BY%20%60created_date%60%20DESC%20NULL%20FIRST'
# headers = {'Ocp-Apim-Subscription-Key': api_key}

In [39]:
# Print the response object to the console
response_services = requests.get(url_service)
print(response.status_code)

200


In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofnewyork.us", None)
# Define date range
start_date = datetime(2023, 1, 1).isoformat()  # January 1, 2023
end_date = datetime(2023, 1, 2).isoformat()  # June 30, 2024
# Example authenticated client (needed for non-public datasets):
client = Socrata(
    "data.cityofnewyork.us",
    token311,
    username= username,
    password= password,
    timeout=1000
)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# Query dataset with date filter
results = client.get("erm2-nwe9", where=f"created_date between '{start_date}' and '{end_date}'",
                     limit=2000)
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)